## Funções:

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

def request():
    res = requests.get('http://127.0.0.1:8000/places/default/sitemap.xml')
    soup = BeautifulSoup(res.text, 'html5lib')
    return soup

In [2]:
def crawler(soup):
    paises = pd.DataFrame()
    links = soup.find_all('loc')
    for link in links:
        tds = {}
        pais = requests.get(link.text)
        soupP = BeautifulSoup(pais.text, 'html5lib')
        trs = soupP.find_all('tr')
        for tr in trs:
            td = tr.find_all('td')
            if td[1].find('img'):
                tds[td[0].text] = td[1].find('img')['src']
            else:
                tds[td[0].text] = td[1].text

                
    df = pd.DataFrame(list(tds.items()), columns=['Country', tds['Country: ']]).set_index('Country').T
    paises = pd.concat([paises, df])
    
    paises.drop('Country: ', axis=1, inplace=True)
    paises.reset_index(inplace=True)
    paises.rename({'index': 'Country'}, axis=1, inplace=True)

    datetime.datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    paises['Timestamp'] = datetime.datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    return paises

In [ ]:
def compara_df(df1,df2):
    df = df2.merge(df1, how = "left")
    return df

## Roda o programa:

In [4]:
Res = request()
crawl = crawler(Res)
crawl.to_csv("data.csv", encoding='latin1', sep=';', index=False)

## Monitoramento das páginas:

In [ ]:
Res = request()
crawl = crawler(Res)
df = pd.read_csv("data.csv")

new_df = compara_df(df, crawl)

new_df.to_csv("data.csv", encoding='latin1', sep=';', index=False)